<a href="https://colab.research.google.com/github/adrianacupp/NLP_with_Transformers/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset**

how emotions are represented in English Twitter messages.
this dataset contains six basic emotions: anger, disgust, fear, joy, sadness, and surprise. Given a tweet, the task will be to train a model that can classify it into one of these emotions.

In [4]:
#!pip install datasets

In [7]:
from datasets import list_datasets

In [11]:
all_datasets = list_datasets()

In [12]:
print(f"There are {len(all_datasets)} datasets currently available on the Hub")
print(f"The first 10 are: {all_datasets[:10]}")

There are 69493 datasets currently available on the Hub
The first 10 are: ['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews']


In [16]:
emotion_datasets = [dataset for dataset in all_datasets if "emotion" in dataset.lower()]

print(len(emotion_datasets))

248


In [14]:
from datasets import load_dataset

In [17]:
emotions = load_dataset("emotion")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]